# 📚 SSG Epub Admin Tool - Google Colab

**Version**: 1.0  
**Updated**: 2024-01-16  
**Purpose**: Automated tool để thêm sách vào SSG Epub Library

## 🎯 Chức năng chính:
- **Mode 1**: Thêm sách manual với platform shortening
- **Mode 2**: Extract từ Google Drive epub file  
- **Mode 3**: Bulk processing từ Google Drive folder
- **Platform Management**: Add/manage URL shortening platforms
- **Legacy Conversion**: Convert existing books với platform mới

## ⚠️ Lưu ý:
- Mode 2 & 3: Cần paste fresh cURL với authentication
- Platform configs được lưu trong GitHub `_data/platforms.yml`
- Tất cả URL được obfuscate trước khi lưu


In [ ]:
# @title 📦 Setup & Installation
# @markdown Cài đặt dependencies và setup môi trường

!pip install PyGithub requests ebooklib cloudinary Pillow python-dotenv pyyaml -q

import os
import json
import yaml
import requests
import base64
import re
import shlex
from datetime import datetime
from urllib.parse import parse_qs, urlparse, urlencode
import time

# Google Colab specific imports
from google.colab import files, drive, userdata
from github import Github

print("✅ Dependencies installed successfully!")
print("🔧 Setup hoàn tất - sẵn sàng sử dụng!")
print("🔐 Secure credential management enabled với Google Colab userdata")


In [ ]:
# @title 🔐 Secure Configuration & Credentials  
# @markdown Load credentials từ Google Colab userdata (secure storage)

class SecureCredentialManager:
    """Quản lý credentials một cách bảo mật với Google Colab userdata"""
    
    def __init__(self):
        self.credentials = {}
        self.load_credentials()
    
    def get_credential(self, key, prompt_text, is_required=True):
        """Get credential từ userdata hoặc prompt user"""
        try:
            # Kiểm tra userdata trước
            value = userdata.get(key)
            if value:
                print(f"🔍 Đã tìm thấy {key} trong userdata")
                return value
        except:
            print(f"⚠️ Không tìm thấy {key} trong userdata")
        
        # Nếu chưa có, yêu cầu người dùng nhập
        if is_required:
            value = input(f"🔑 {prompt_text}: ").strip()
            if value:
                print(f"📝 Bạn có thể lưu vào userdata: userdata.set('{key}', 'your_value')")
                return value
            else:
                print(f"❌ {key} is required!")
                return None
        else:
            value = input(f"🔑 {prompt_text} (optional): ").strip()
            if value:
                print(f"📝 Có thể lưu vào userdata: userdata.set('{key}', 'your_value')")
            return value or None
    
    def load_credentials(self):
        """Load tất cả credentials cần thiết"""
        print("🔐 Loading Secure Credentials")
        print("=" * 40)
        
        # GitHub credentials (required)
        self.credentials['github_token'] = self.get_credential(
            'GITHUB_TOKEN', 
            'GitHub Personal Access Token', 
            True
        )
        
        self.credentials['github_repo'] = self.get_credential(
            'GITHUB_REPO', 
            'GitHub Repository (user/repo)', 
            True
        )
        
        self.credentials['branch_name'] = self.get_credential(
            'BRANCH_NAME', 
            'Git Branch Name', 
            False
        ) or "main"
        
        # Cloudinary credentials (optional)
        self.credentials['cloudinary_cloud_name'] = self.get_credential(
            'CLOUDINARY_CLOUD_NAME', 
            'Cloudinary Cloud Name', 
            False
        )
        
        if self.credentials['cloudinary_cloud_name']:
            self.credentials['cloudinary_api_key'] = self.get_credential(
                'CLOUDINARY_API_KEY', 
                'Cloudinary API Key', 
                False
            )
            
            self.credentials['cloudinary_api_secret'] = self.get_credential(
                'CLOUDINARY_API_SECRET', 
                'Cloudinary API Secret', 
                False
            )
        
        # API Configuration
        self.credentials['default_config_api'] = self.get_credential(
            'DEFAULT_CONFIG_API', 
            'Default Config API URL', 
            False
        ) or ""
        
        # Validate and test GitHub
        self.test_github_connection()
    
    def test_github_connection(self):
        """Test GitHub connection"""
        if not self.credentials['github_token'] or not self.credentials['github_repo']:
            print("❌ GitHub credentials missing!")
            return False
        
        try:
            g = Github(self.credentials['github_token'])
            repo = g.get_repo(self.credentials['github_repo'])
            print(f"✅ GitHub connection successful: {repo.full_name}")
            print(f"📊 Repository stats: {repo.stargazers_count} stars, {repo.forks_count} forks")
            
            self.credentials['github_instance'] = g
            self.credentials['repo_instance'] = repo
            return True
        except Exception as e:
            print(f"❌ GitHub connection failed: {e}")
            print("🔧 Please check your token and repository name")
            return False
    
    def get(self, key):
        """Get credential value"""
        return self.credentials.get(key)
    
    def show_status(self):
        """Show configuration status"""
        print("\n🔧 Configuration Status:")
        print("=" * 30)
        print(f"📂 Target repository: {self.get('github_repo')}")
        print(f"🌿 Target branch: {self.get('branch_name')}")
        
        if self.get('cloudinary_cloud_name'):
            print(f"☁️ Cloudinary configured: {self.get('cloudinary_cloud_name')}")
        else:
            print("⚠️ Cloudinary not configured (image processing disabled)")
        
        if self.get('default_config_api'):
            print(f"🔗 Default API: {self.get('default_config_api')[:50]}...")
        
        print("\n💡 Tip: Lưu credentials vào userdata để tránh nhập lại:")
        print("   userdata.set('GITHUB_TOKEN', 'your_token')")
        print("   userdata.set('GITHUB_REPO', 'user/repo')")

# Initialize secure credential manager
credential_manager = SecureCredentialManager()
credential_manager.show_status()


In [ ]:
# @title 🔧 Core Classes - GitHub Manager
# @markdown GitHub operations for file management

class GitHubManager:
    def __init__(self, token, repo_name, branch="main"):
        self.github = Github(token)
        self.repo = self.github.get_repo(repo_name)
        self.branch = branch
        
    def file_exists(self, file_path):
        """Check if file exists in repo"""
        try:
            self.repo.get_contents(file_path, ref=self.branch)
            return True
        except:
            return False
    
    def get_file_content(self, file_path):
        """Get file content as string"""
        try:
            file_data = self.repo.get_contents(file_path, ref=self.branch)
            content = base64.b64decode(file_data.content).decode('utf-8')
            return content
        except Exception as e:
            print(f"❌ Failed to get file content: {e}")
            return None
    
    def create_file(self, file_path, content, commit_message):
        """Create new file in repository"""
        try:
            if self.file_exists(file_path):
                raise Exception(f"File {file_path} already exists!")
            
            self.repo.create_file(
                path=file_path,
                message=commit_message,
                content=content,
                branch=self.branch
            )
            return True
        except Exception as e:
            print(f"❌ Failed to create file: {e}")
            return False
    
    def update_file(self, file_path, content, commit_message):
        """Update existing file"""
        try:
            file_data = self.repo.get_contents(file_path, ref=self.branch)
            self.repo.update_file(
                path=file_path,
                message=commit_message,
                content=content,
                sha=file_data.sha,
                branch=self.branch
            )
            return True
        except Exception as e:
            print(f"❌ Failed to update file: {e}")
            return False
    
    def check_duplicate_book(self, title, author):
        """Check if book already exists"""
        try:
            contents = self.repo.get_contents("_epubs", ref=self.branch)
            
            for content in contents:
                if content.name.endswith('.md'):
                    file_content = base64.b64decode(content.content).decode('utf-8')
                    
                    if '---' in file_content:
                        yaml_content = file_content.split('---')[1]
                        if f'title: "{title}"' in yaml_content and f'author: "{author}"' in yaml_content:
                            return True, content.name
            
            return False, None
        except Exception as e:
            print(f"⚠️ Could not check duplicates: {e}")
            return False, None

# Initialize GitHub manager với secure credentials
if credential_manager.get('github_token') and credential_manager.get('github_repo'):
    try:
        github_manager = GitHubManager(
            credential_manager.get('github_token'), 
            credential_manager.get('github_repo'), 
            credential_manager.get('branch_name')
        )
        print("📂 GitHub Manager initialized successfully!")
    except Exception as e:
        print(f"❌ GitHub Manager failed: {e}")
        github_manager = None
else:
    github_manager = None
    print("⚠️ GitHub Manager not initialized - please configure credentials first")


In [ ]:
# @title 🔗 URL Obfuscation System
# @markdown Simple obfuscation cho download URLs

class LinkObfuscator:
    """Simple obfuscation utilities - consistent với frontend"""
    
    @staticmethod
    def encode(url):
        try:
            # Simple XOR with key + Base64 (same as frontend)
            key = 'SSGEpub2024'
            result = ''
            for i in range(len(url)):
                result += chr(ord(url[i]) ^ ord(key[i % len(key)]))
            
            encoded = base64.b64encode(result.encode('utf-8')).decode('utf-8')
            return f"data:encoded,{encoded}"
        except Exception as e:
            print(f"⚠️ Failed to encode URL: {e}")
            return url
    
    @staticmethod
    def decode(encoded_url):
        try:
            if encoded_url.startswith('data:encoded,'):
                encoded_part = encoded_url.replace('data:encoded,', '')
            else:
                encoded_part = encoded_url
            
            key = 'SSGEpub2024'
            decoded = base64.b64decode(encoded_part).decode('utf-8')
            result = ''
            for i in range(len(decoded)):
                result += chr(ord(decoded[i]) ^ ord(key[i % len(key)]))
            
            return result
        except Exception as e:
            print(f"⚠️ Failed to decode URL: {e}")
            return None

# Test obfuscation
test_url = "https://drive.google.com/file/d/1atVRKbZ07rSF5X_Q0OW1lLKywqAj4yvd/view?usp=sharing"
encoded = LinkObfuscator.encode(test_url)
decoded = LinkObfuscator.decode(encoded)

print("🧪 Testing URL Obfuscation:")
print(f"Original: {test_url[:50]}...")
print(f"Encoded: {encoded[:50]}...")
print(f"Decoded: {decoded[:50]}...")
print(f"✅ Test: {'PASSED' if decoded == test_url else 'FAILED'}")

link_obfuscator = LinkObfuscator()
print("🔗 Link Obfuscator ready!")


In [ ]:
# @title 🌐 Dynamic Platform Manager
# @markdown Universal cURL parser cho custom shortening platforms

class DynamicPlatformManager:
    """Dynamic platform manager với simplified authentication"""
    
    def __init__(self):
        self.platforms = {}
        
    def parse_curl_command(self, curl_command):
        """Parse cURL command thành request components"""
        try:
            # Clean command
            curl_command = curl_command.strip()
            if curl_command.startswith('curl '):
                curl_command = curl_command[5:]
            
            config = {
                'method': 'GET',
                'url': '',
                'headers': {},
                'data': None,
                'json': None
            }
            
            # Extract method
            method_match = re.search(r'-X\s+(\w+)', curl_command)
            if method_match:
                config['method'] = method_match.group(1).upper()
            
            # Extract URL
            urls = re.findall(r'https?://[^\s\'"]+', curl_command)
            if urls:
                config['url'] = urls[0]
            
            # Extract headers
            headers = re.findall(r'-H\s+[\'"]([^\'"]*)[\'"]', curl_command)
            for header in headers:
                if ':' in header:
                    key, value = header.split(':', 1)
                    config['headers'][key.strip()] = value.strip()
            
            # Extract data
            data_matches = re.findall(r'-d\s+[\'"]([^\'"]*)[\'"]', curl_command)
            if data_matches:
                data_string = data_matches[0]
                
                # Determine if JSON or form data
                content_type = config['headers'].get('Content-Type', '').lower()
                if 'application/json' in content_type:
                    try:
                        config['json'] = json.loads(data_string)
                    except:
                        config['data'] = data_string
                else:
                    # Parse as form data
                    config['data'] = self._parse_form_data(data_string)
            
            return config
            
        except Exception as e:
            print(f"❌ Error parsing cURL: {e}")
            return None
    
    def _parse_form_data(self, data_string):
        """Parse form data string"""
        try:
            result = {}
            pairs = data_string.split('&')
            for pair in pairs:
                if '=' in pair:
                    key, value = pair.split('=', 1)
                    result[key] = value
            return result
        except:
            return data_string
    
    def add_platform_from_curl(self, platform_name, curl_command, response_config=None):
        """Add platform từ cURL example"""
        request_config = self.parse_curl_command(curl_command)
        if not request_config:
            print(f"❌ Failed to parse cURL for {platform_name}")
            return False
        
        if not response_config:
            response_config = {'type': 'auto'}
        
        self.platforms[platform_name] = {
            'name': platform_name,
            'request_config': request_config,
            'response_config': response_config,
            'curl_template': curl_command
        }
        
        print(f"✅ Added platform: {platform_name}")
        return True
    
    def shorten_url(self, platform_name, target_url):
        """Shorten URL với specified platform"""
        if platform_name not in self.platforms:
            print(f"❌ Platform not found: {platform_name}")
            return target_url
        
        platform = self.platforms[platform_name]
        
        try:
            # Build request với target URL
            request_config = self._prepare_request(platform['request_config'], target_url)
            
            # Execute request
            response = self._execute_request(request_config)
            
            if response and response.status_code == 200:
                short_url = self._extract_short_url(response, platform['response_config'])
                
                if short_url and short_url.startswith('http'):
                    print(f"✅ {platform_name}: {target_url[:50]}... → {short_url}")
                    return short_url
                else:
                    print(f"⚠️ Could not extract URL from {platform_name} response")
                    return target_url
            else:
                status = response.status_code if response else 'No response'
                print(f"❌ {platform_name} request failed: {status}")
                return target_url
                
        except Exception as e:
            print(f"❌ Error with {platform_name}: {e}")
            return target_url
    
    def _prepare_request(self, config, target_url):
        """Replace ${link_drive} placeholder với actual URL"""
        prepared_config = config.copy()
        
        # Replace in JSON data
        if prepared_config.get('json'):
            prepared_config['json'] = self._replace_placeholder(
                prepared_config['json'], target_url
            )
        
        # Replace in form data
        if prepared_config.get('data'):
            prepared_config['data'] = self._replace_placeholder(
                prepared_config['data'], target_url
            )
        
        return prepared_config
    
    def _replace_placeholder(self, data, target_url):
        """Replace ${link_drive} placeholder recursively"""
        if isinstance(data, dict):
            return {k: self._replace_placeholder(v, target_url) for k, v in data.items()}
        elif isinstance(data, str):
            return data.replace('${link_drive}', target_url)
        else:
            return data
    
    def _execute_request(self, config):
        """Execute HTTP request"""
        try:
            method = config['method'].lower()
            url = config['url']
            headers = config.get('headers', {})
            
            kwargs = {'headers': headers}
            
            if config.get('json'):
                kwargs['json'] = config['json']
            elif config.get('data'):
                kwargs['data'] = config['data']
            
            response = getattr(requests, method)(url, **kwargs)
            return response
            
        except Exception as e:
            print(f"⚠️ Request execution failed: {e}")
            return None
    
    def _extract_short_url(self, response, config):
        """Extract short URL từ response"""
        try:
            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
                
                if 'path' in config:
                    path_parts = config['path'].split('.')
                    current = data
                    for part in path_parts:
                        if isinstance(current, dict) and part in current:
                            current = current[part]
                        else:
                            return None
                    return str(current) if current else None
                
                # Auto-detect common fields
                common_fields = ['short_url', 'shortened_url', 'url', 'link', 'shortlink']
                for field in common_fields:
                    if field in data:
                        return str(data[field])
            else:
                text = response.text.strip()
                if 'regex' in config:
                    match = re.search(config['regex'], text)
                    return match.group(0) if match else None
                elif text.startswith('http'):
                    return text
            
            return None
            
        except Exception as e:
            print(f"⚠️ Extraction error: {e}")
            return None

# Initialize dynamic platform manager
dynamic_platform_manager = DynamicPlatformManager()
print("🌐 Dynamic Platform Manager initialized!")


In [ ]:
# @title 🏗️ Platform Configuration Manager
# @markdown Quản lý platform configs trong GitHub

class PlatformConfigManager:
    def __init__(self, github_manager):
        self.github_manager = github_manager
        self.config_file = "_data/platforms.yml"
        self.platforms = {}
        self.next_index = 0
        
        # Load existing platforms
        self.load_platforms()
    
    def load_platforms(self):
        """Load platforms từ GitHub"""
        try:
            if self.github_manager and self.github_manager.file_exists(self.config_file):
                content = self.github_manager.get_file_content(self.config_file)
                if content:
                    config = yaml.safe_load(content)
                    if config and 'platforms' in config:
                        for platform in config['platforms']:
                            self.platforms[platform['id']] = platform
                        
                        # Calculate next available index
                        if self.platforms:
                            self.next_index = max(p['index'] for p in self.platforms.values()) + 1
                        else:
                            self.next_index = 0
                        
                        print(f"✅ Loaded {len(self.platforms)} platforms from GitHub")
                        return
            
            # Initialize with default platforms
            self._initialize_default_platforms()
            
        except Exception as e:
            print(f"⚠️ Error loading platforms: {e}")
            self._initialize_default_platforms()
    
    def _initialize_default_platforms(self):
        """Initialize với default platforms"""
        default_platforms = [
            {
                'name': 'Google Drive',
                'id': 'gdrive',
                'index': 0,
                'icon': 'fab fa-google-drive',
                'type': 'direct',
                'active': True,
                'created_date': datetime.now().isoformat()
            },
            {
                'name': 'OneDrive',
                'id': 'onedrive',
                'index': 1,
                'icon': 'fab fa-microsoft',
                'type': 'direct',
                'active': True,
                'created_date': datetime.now().isoformat()
            }
        ]
        
        for platform in default_platforms:
            self.platforms[platform['id']] = platform
        
        self.next_index = 2
        print("🔧 Initialized với default platforms")
    
    def add_platform(self, name, platform_id, icon, curl_template, response_config):
        """Add new platform với fixed index"""
        try:
            if platform_id in self.platforms:
                print(f"⚠️ Platform {platform_id} already exists!")
                return False
            
            new_platform = {
                'name': name,
                'id': platform_id,
                'index': self.next_index,
                'icon': icon,
                'type': 'shortener',
                'active': True,
                'created_date': datetime.now().isoformat(),
                'curl_template': curl_template,
                'response_config': response_config
            }
            
            self.platforms[platform_id] = new_platform
            self.next_index += 1
            
            # Save to GitHub
            if self.save_platforms():
                print(f"✅ Added platform: {name} (index {new_platform['index']})\")\n                return True\n            else:\n                # Rollback\n                del self.platforms[platform_id]\n                self.next_index -= 1\n                print(f\"❌ Failed to save platform {name}\")\n                return False\n                \n        except Exception as e:\n            print(f\"❌ Error adding platform: {e}\")\n            return False\n    \n    def save_platforms(self):\n        \"\"\"Save platforms to GitHub\"\"\"\n        try:\n            config = {\n                'platforms': list(self.platforms.values()),\n                'metadata': {\n                    'version': '1.0.0',\n                    'last_updated': datetime.now().isoformat(),\n                    'next_index': self.next_index\n                }\n            }\n            \n            yaml_content = yaml.dump(config, default_flow_style=False, allow_unicode=True)\n            \n            if self.github_manager.file_exists(self.config_file):\n                success = self.github_manager.update_file(\n                    self.config_file,\n                    yaml_content,\n                    f\"Cập nhật platform configurations - {len(self.platforms)} platforms\"\n                )\n            else:\n                success = self.github_manager.create_file(\n                    self.config_file,\n                    yaml_content,\n                    \"Khởi tạo platform configurations\"\n                )\n            \n            if success:\n                print(f\"💾 Saved {len(self.platforms)} platforms to GitHub\")\n                return True\n            else:\n                print(\"❌ Failed to save platforms to GitHub\")\n                return False\n                \n        except Exception as e:\n            print(f\"❌ Error saving platforms: {e}\")\n            return False\n    \n    def get_platform_by_index(self, index):\n        \"\"\"Get platform by index\"\"\"\n        for platform in self.platforms.values():\n            if platform['index'] == index:\n                return platform\n        return None\n    \n    def list_platforms(self):\n        \"\"\"List all platforms\"\"\"\n        print(\"📋 Available Platforms:\")\n        print(\"=\" * 50)\n        \n        sorted_platforms = sorted(self.platforms.values(), key=lambda x: x['index'])\n        \n        for platform in sorted_platforms:\n            status = \"🟢\" if platform['active'] else \"🔴\"\n            platform_type = \"📎\" if platform['type'] == 'direct' else \"🔗\"\n            \n            print(f\"{status} {platform_type} [{platform['index']}] {platform['name']} ({platform['id']})\")\n            \n            if platform.get('curl_template'):\n                print(f\"   📝 cURL configured\")\n            \n        print(f\"\\n📊 Total: {len(self.platforms)} platforms\")\n        print(f\"🔢 Next index: {self.next_index}\")\n\n# Initialize platform config manager\nif github_manager:\n    platform_config_manager = PlatformConfigManager(github_manager)\n    print(\"🏗️ Platform Config Manager initialized!\")\nelse:\n    platform_config_manager = None\n    print(\"⚠️ Platform Config Manager not initialized - GitHub required\")"


In [ ]:
# @title 📝 Markdown Generator
# @markdown Generate Jekyll-compatible markdown files

class MarkdownGenerator:
    def __init__(self):
        self.obfuscator = LinkObfuscator()
    
    def generate_filename(self, title):
        """Generate filename from title"""
        # Remove Vietnamese tones and special chars
        filename = self.remove_vietnamese_tones(title)
        filename = re.sub(r'[^a-zA-Z0-9\s]', '', filename)
        filename = re.sub(r'\s+', '-', filename.strip().lower())
        return f"{filename[:50]}.md"
    
    def remove_vietnamese_tones(self, text):
        """Remove Vietnamese accents"""
        replacements = {
            'à|á|ạ|ả|ã|â|ầ|ấ|ậ|ẩ|ẫ|ă|ằ|ắ|ặ|ẳ|ẵ': 'a',
            'è|é|ẹ|ẻ|ẽ|ê|ề|ế|ệ|ể|ễ': 'e',
            'ì|í|ị|ỉ|ĩ': 'i',
            'ò|ó|ọ|ỏ|õ|ô|ồ|ố|ộ|ổ|ỗ|ơ|ờ|ớ|ợ|ở|ỡ': 'o',
            'ù|ú|ụ|ủ|ũ|ư|ừ|ứ|ự|ử|ữ': 'u',
            'ỳ|ý|ỵ|ỷ|ỹ': 'y',
            'đ': 'd'
        }
        
        for pattern, replacement in replacements.items():
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        
        return text
    
    def generate_markdown(self, book_data):
        """Generate complete markdown file content"""
        
        yaml_lines = ["---", "layout: epub"]
        
        # Required fields
        yaml_lines.append(f'title: "{book_data["title"]}"')
        yaml_lines.append(f'author: "{book_data["author"]}"')
        yaml_lines.append(f'cover_image: "{book_data["cover_image"]}"')
        
        # Optional fields
        optional_fields = ['preview_image', 'isbn', 'published_date', 'language', 'publisher']
        for field in optional_fields:
            if book_data.get(field):
                yaml_lines.append(f'{field}: "{book_data[field]}"')
        
        # Genre array
        if book_data.get("genre"):
            yaml_lines.append("genre:")
            for genre in book_data["genre"]:
                yaml_lines.append(f'  - "{genre}"')
        
        yaml_lines.append(f'description: "{book_data["description"]}"')
        
        # Numeric fields
        if book_data.get("rating"):
            yaml_lines.append(f'rating: {book_data["rating"]}')
        
        if book_data.get("pages"):
            yaml_lines.append(f'pages: {book_data["pages"]}')
        
        # Preview content
        if book_data.get("preview_content"):
            yaml_lines.append("preview_content: |")
            for line in book_data["preview_content"].split('\n'):
                yaml_lines.append(f"  {line}")
        
        # Download links với obfuscation
        if book_data.get("download_links"):
            yaml_lines.append("download_links:")
            for link in book_data["download_links"]:
                # Obfuscate URL if not already obfuscated
                url = link["url"]
                if not url.startswith('data:encoded,'):
                    url = self.obfuscator.encode(url)
                
                yaml_lines.append(f'  - platform: "{link["platform"]}"')
                yaml_lines.append(f'    url: "{url}"')
                yaml_lines.append(f'    index: {link["index"]}')
                yaml_lines.append(f'    icon: "{link["icon"]}"')
        
        # Config URL
        if book_data.get("config_url"):
            yaml_lines.append(f'download_config_url: "{book_data["config_url"]}"')
        
        # Tags
        if book_data.get("tags"):
            yaml_lines.append("tags:")
            for tag in book_data["tags"]:
                yaml_lines.append(f'  - "{tag}"')
        
        yaml_lines.append("---")
        yaml_lines.append("")
        yaml_lines.append(f'Đây là trang chi tiết của cuốn sách "{{{{ page.title }}}}" của tác giả {{{{ page.author }}}}.') 
        
        return '\n'.join(yaml_lines)

# Initialize markdown generator
markdown_generator = MarkdownGenerator()
print("📝 Markdown Generator initialized!")


In [ ]:
# @title 🚀 System Status Check
# @markdown Kiểm tra status và sẵn sàng sử dụng

def check_system_status():
    """Check system components status"""
    print("🔍 System Status Check:")
    print("=" * 40)
    
    # GitHub
    if github_manager:
        print("✅ GitHub Manager: Ready")
        try:
            # Test repo access
            repo_info = github_manager.repo
            print(f"   📂 Repository: {repo_info.full_name}")
            print(f"   🌿 Branch: {github_manager.branch}")
        except Exception as e:
            print(f"   ⚠️ Repository access issue: {e}")
    else:
        print("❌ GitHub Manager: Not configured")
    
    # Platform Manager
    print(f"✅ Platform Manager: Ready ({len(dynamic_platform_manager.platforms)} platforms)")
    
    # Platform Config Manager
    if platform_config_manager:
        print(f"✅ Platform Config: Ready ({len(platform_config_manager.platforms)} registered)")
    else:
        print("❌ Platform Config: Not initialized")
    
    # Obfuscation
    print("✅ Link Obfuscation: Ready")
    
    # Markdown Generator
    print("✅ Markdown Generator: Ready")
    
    # Cloudinary
    if credential_manager.get('cloudinary_cloud_name'):
        print(f"✅ Cloudinary: Configured ({credential_manager.get('cloudinary_cloud_name')})")
    else:
        print("⚠️ Cloudinary: Not configured (image processing disabled)")
    
    print("\n🎯 Available Commands:")
    print("   📚 Mode 1: Manual book addition")
    print("   📂 Mode 2: Google Drive epub extraction")
    print("   📁 Mode 3: Bulk folder processing")
    print("   🔧 Platform Management")
    print("   🔄 Legacy Conversion")

# Run status check
check_system_status()

print("\n🚀 SSG Epub Admin Tool sẵn sàng sử dụng!")
print("📋 Chạy các cell bên dưới để bắt đầu...")


---
# 🔧 Platform Management

Quản lý các platform rút gọn link


In [ ]:
# @title 📋 List Existing Platforms
# @markdown Hiển thị tất cả platforms đã đăng ký

if platform_config_manager:
    platform_config_manager.list_platforms()
else:
    print("❌ Platform Config Manager not available!")
    print("🔧 Please configure GitHub credentials first")


In [ ]:
# @title ➕ Add New Platform
# @markdown Thêm platform rút gọn mới

def add_new_platform():
    """Interactive platform addition"""
    if not platform_config_manager:
        print("❌ Platform Config Manager not available!")
        return
    
    print("🔧 Thêm Platform Rút Gọn Mới")
    print("=" * 40)
    
    # Platform info
    platform_name = input("📝 Tên platform (VD: YeuMoney): ").strip()
    if not platform_name:
        print("❌ Platform name is required!")
        return
    
    platform_id = input("🆔 Platform ID (VD: yeumoney): ").strip().lower()
    if not platform_id:
        platform_id = platform_name.lower().replace(' ', '')
    
    icon = input("🎨 Icon class (VD: fas fa-heart): ").strip()
    if not icon:
        icon = "fas fa-link"
    
    print(f"\n📝 Nhập cURL command (có chứa ${{link_drive}}):")
    print("VD: curl -X POST 'https://yeumoney.com/api' -H 'Authorization: Bearer abc123' -d '{\"url\": \"${{link_drive}}\"}'\")
    
    curl_command = input("\ncURL: ").strip()
    if not curl_command:
        print("❌ cURL command is required!")
        return
    
    # Response config
    print(f"\n🔧 Response Configuration:")
    print("1. Auto-detect (recommended)")
    print("2. JSON path (VD: data.short_url)")
    print("3. Regex pattern")
    
    config_choice = input("Chọn (1-3): ").strip()
    
    response_config = {'type': 'auto'}
    
    if config_choice == "2":
        json_path = input("JSON path (VD: data.short_url): ").strip()
        if json_path:
            response_config = {'type': 'json', 'path': json_path}
    elif config_choice == "3":
        regex_pattern = input("Regex pattern: ").strip()
        if regex_pattern:
            response_config = {'type': 'text', 'regex': regex_pattern}
    
    # Test platform
    print(f"\n🧪 Testing platform configuration...")
    
    # Add to dynamic manager for testing
    dynamic_platform_manager.add_platform_from_curl(platform_name, curl_command, response_config)
    
    test_url = "https://drive.google.com/file/d/1atVRKbZ07rSF5X_Q0OW1lLKywqAj4yvd/view?usp=sharing"
    print(f"🔗 Testing với: {test_url[:50]}...")
    
    result = dynamic_platform_manager.shorten_url(platform_name, test_url)
    
    if result != test_url:
        print(f"✅ Test successful: {result}")
        
        # Confirm save
        save_confirm = input(f"\n💾 Save platform '{platform_name}' to GitHub? (y/N): ").strip().lower()
        
        if save_confirm == 'y':
            success = platform_config_manager.add_platform(
                platform_name, platform_id, icon, curl_command, response_config
            )
            
            if success:
                print(f"✅ Platform '{platform_name}' added successfully!")
                print(f"📍 Assigned index: {platform_config_manager.platforms[platform_id]['index']}")
            else:
                print(f"❌ Failed to save platform!")
        else:
            print("❌ Platform not saved")
    else:
        print(f"❌ Test failed - platform không hoạt động đúng")
        print("🔧 Kiểm tra lại cURL command và response config")

# Run interactive platform addition
add_new_platform()


---
# 📚 Mode 1: Manual Book Addition  

Thêm sách thủ công với platform shortening


In [ ]:
# @title 📝 Mode 1: Manual Book Addition
# @markdown Thêm sách thủ công với platform shortening

def mode1_manual_book_addition():
    """Mode 1: Manual book addition với platform shortening"""
    
    if not github_manager or not platform_config_manager:
        print("❌ Required managers not available!")
        print("🔧 Please configure GitHub credentials first")
        return
    
    print("📚 Mode 1: Manual Book Addition")
    print("=" * 50)
    
    # Basic book information
    print("📖 Basic Information:")
    title = input("📖 Title: ").strip()
    if not title:
        print("❌ Title is required!")
        return
    
    author = input("✍️ Author: ").strip()
    if not author:
        print("❌ Author is required!")
        return
    
    # Check for duplicates
    print(f"\n🔍 Checking for duplicates...")
    is_duplicate, existing_file = github_manager.check_duplicate_book(title, author)
    
    if is_duplicate:
        print(f"⚠️ Book already exists: {existing_file}")
        overwrite = input("📝 Continue anyway? (y/N): ").strip().lower()
        if overwrite != 'y':
            print("❌ Operation cancelled")
            return
    
    # Collect book metadata
    description = input("📄 Description: ").strip()
    cover_image = input("🖼️ Cover image URL: ").strip()
    
    # Optional fields
    preview_image = input("🖼️ Preview image URL (optional): ").strip()
    isbn = input("📊 ISBN (optional): ").strip()
    published_date = input("📅 Published date (YYYY-MM-DD, optional): ").strip()
    language = input("🌐 Language (optional): ").strip() or "Tiếng Việt"
    publisher = input("🏢 Publisher (optional): ").strip()
    
    # Genre
    genres_input = input("🏷️ Genres (comma separated): ").strip()
    genres = [g.strip() for g in genres_input.split(',')] if genres_input else []
    
    # Rating and pages
    rating_input = input("⭐ Rating (1-5, optional): ").strip()
    rating = float(rating_input) if rating_input and rating_input.replace('.', '').isdigit() else None
    
    pages_input = input("📄 Pages (optional): ").strip()
    pages = int(pages_input) if pages_input.isdigit() else None
    
    # Config URL
    config_url = input("🔗 Config API URL (optional): ").strip() or credential_manager.get('default_config_api')
    
    # Download links setup
    print(f"\n🔗 Download Links Setup:")
    print("Available platforms:")
    if platform_config_manager:
        platform_config_manager.list_platforms()
    
    download_links = []
    
    while True:
        print(f"\n➕ Adding download link #{len(download_links) + 1}")
        
        # Show available platforms
        available_platforms = list(platform_config_manager.platforms.keys())
        print(f"Available platforms: {', '.join(available_platforms)}")
        
        platform_id = input("🌐 Platform ID (hoặc 'done' để hoàn thành): ").strip().lower()
        
        if platform_id == 'done':
            break
        
        if platform_id not in platform_config_manager.platforms:
            print(f"❌ Platform '{platform_id}' not found!")
            continue
        
        platform_info = platform_config_manager.platforms[platform_id]
        
        # Get original URL
        original_url = input(f"🔗 Original URL cho {platform_info['name']}: ").strip()
        if not original_url:
            print("❌ URL is required!")
            continue
        
        # For shortener platforms, get cURL for shortening
        if platform_info['type'] == 'shortener':
            print(f"\n🔧 Shortening with {platform_info['name']}...")
            print(f"📝 Paste fresh cURL command (với authentication):\")\n            print(f\"(Replace ${{link_drive}} sẽ được thay thế tự động)\")\n            \n            fresh_curl = input(\"cURL: \").strip()\n            if fresh_curl:\n                # Update platform with fresh cURL\n                dynamic_platform_manager.add_platform_from_curl(\n                    platform_info['name'], \n                    fresh_curl, \n                    platform_info.get('response_config')\n                )\n                \n                # Shorten URL\n                shortened_url = dynamic_platform_manager.shorten_url(platform_info['name'], original_url)\n                \n                if shortened_url != original_url:\n                    final_url = shortened_url\n                    print(f\"✅ Shortened: {original_url[:30]}... → {shortened_url}\")\n                else:\n                    print(f\"⚠️ Shortening failed, using original URL\")\n                    final_url = original_url\n            else:\n                print(f\"⚠️ No cURL provided, using original URL\")\n                final_url = original_url\n        else:\n            # Direct platform\n            final_url = original_url\n        \n        download_links.append({\n            'platform': platform_info['name'],\n            'url': final_url,\n            'index': platform_info['index'],\n            'icon': platform_info['icon']\n        })\n        \n        print(f\"✅ Added {platform_info['name']} (index {platform_info['index']})\")\n    \n    if not download_links:\n        print(\"❌ At least one download link is required!\")\n        return\n    \n    # Build book data\n    book_data = {\n        'title': title,\n        'author': author,\n        'description': description,\n        'cover_image': cover_image,\n        'download_links': download_links,\n        'config_url': config_url\n    }\n    \n    # Add optional fields\n    if preview_image:\n        book_data['preview_image'] = preview_image\n    if isbn:\n        book_data['isbn'] = isbn\n    if published_date:\n        book_data['published_date'] = published_date\n    if language:\n        book_data['language'] = language\n    if publisher:\n        book_data['publisher'] = publisher\n    if genres:\n        book_data['genre'] = genres\n    if rating:\n        book_data['rating'] = rating\n    if pages:\n        book_data['pages'] = pages\n    \n    # Generate markdown\n    print(f\"\\n📝 Generating markdown file...\")\n    markdown_content = markdown_generator.generate_markdown(book_data)\n    filename = markdown_generator.generate_filename(title)\n    \n    print(f\"📄 Generated file: {filename}\")\n    print(f\"📏 Content length: {len(markdown_content)} characters\")\n    \n    # Preview\n    print(f\"\\n📋 Preview (first 300 chars):\")\n    print(\"-\" * 40)\n    print(markdown_content[:300] + \"...\")\n    print(\"-\" * 40)\n    \n    # Confirm upload\n    upload_confirm = input(f\"\\n📤 Upload '{filename}' to GitHub? (y/N): \").strip().lower()\n    \n    if upload_confirm == 'y':\n        file_path = f\"_epubs/{filename}\"\n        commit_message = f\"Thêm sách: {title} - {author}\"\n        \n        try:\n            if github_manager.file_exists(file_path):\n                success = github_manager.update_file(file_path, markdown_content, commit_message)\n                action = \"Updated\"\n            else:\n                success = github_manager.create_file(file_path, markdown_content, commit_message)\n                action = \"Created\"\n            \n            if success:\n                print(f\"✅ {action} file successfully!\")\n                print(f\"📂 File: {file_path}\")\n                print(f\"💬 Commit: {commit_message}\")\n                print(f\"🚀 GitHub Pages sẽ rebuild automatically\")\n                \n                # Summary\n                print(f\"\\n📊 Summary:\")\n                print(f\"   📖 Title: {title}\")\n                print(f\"   ✍️ Author: {author}\")\n                print(f\"   🔗 Download links: {len(download_links)}\")\n                for link in download_links:\n                    print(f\"      [{link['index']}] {link['platform']}\")\n            else:\n                print(f\"❌ Failed to {action.lower()} file!\")\n                \n        except Exception as e:\n            print(f\"❌ Error uploading file: {e}\")\n    else:\n        print(\"❌ File not uploaded\")\n        print(f\"💾 Markdown content saved locally for reference\")\n\n# Run Mode 1\nmode1_manual_book_addition()"


In [ ]:
# @title 🔐 Secure Platform Authentication Manager
# @markdown Quản lý authentication cho các platform một cách bảo mật

class SecurePlatformAuthManager:
    """Quản lý authentication cho platforms với userdata"""
    
    def __init__(self, credential_manager):
        self.credential_manager = credential_manager
        self.platform_auth = {}
    
    def get_platform_auth(self, platform_id, platform_name, auth_type="cookie"):
        """Get authentication cho platform từ userdata hoặc input"""
        auth_key = f"{platform_id}_{auth_type}".upper()
        
        try:
            # Kiểm tra userdata trước
            auth_value = userdata.get(auth_key)
            if auth_value:
                print(f"🔍 Đã tìm thấy {auth_type} cho {platform_name} trong userdata")
                return auth_value
        except:
            print(f"⚠️ Không tìm thấy {auth_type} cho {platform_name} trong userdata")
        
        # Yêu cầu user nhập
        if auth_type == "cookie":
            prompt = f"🍪 Nhập cookie cho {platform_name}"
        elif auth_type == "token":
            prompt = f"🔑 Nhập access token cho {platform_name}"
        else:
            prompt = f"🔐 Nhập {auth_type} cho {platform_name}"
        
        auth_value = input(f"{prompt}: ").strip()
        
        if auth_value:
            print(f"📝 Lưu vào userdata: userdata.set('{auth_key}', 'your_{auth_type}')")
            self.platform_auth[platform_id] = {
                'type': auth_type,
                'value': auth_value
            }
            return auth_value
        else:
            print(f"❌ {auth_type} cho {platform_name} không được cung cấp")
            return None
    
    def get_fresh_curl_with_auth(self, platform_id, platform_name, base_curl_template):
        """Get fresh cURL với authentication mới"""
        print(f"\n🔧 Cần fresh authentication cho {platform_name}")
        print("⚠️ Platform authentication có thể đã hết hạn")
        print("📝 Vui lòng cung cấp cURL command mới với valid authentication")
        print("🔗 Placeholder ${link_drive} sẽ được thay thế tự động")
        print("\nVí dụ:")
        print(f"curl -X POST 'https://{platform_id}.com/api' \\")
        print("  -H 'Authorization: Bearer your_fresh_token' \\")
        print("  -d '{\"url\": \"${link_drive}\"}'")
        
        fresh_curl = input(f"\n🔄 Fresh cURL cho {platform_name}: ").strip()
        
        if fresh_curl:
            print(f"✅ Fresh cURL received cho {platform_name}")
            
            # Extract và lưu authentication info nếu có thể
            if 'Authorization: Bearer' in fresh_curl:
                auth_match = re.search(r'Authorization: Bearer ([^\'\"\\s]+)', fresh_curl)
                if auth_match:
                    token = auth_match.group(1)
                    auth_key = f"{platform_id}_TOKEN".upper()
                    print(f"🔑 Extracted token - có thể lưu: userdata.set('{auth_key}', '{token[:10]}...')")
            
            elif 'Cookie:' in fresh_curl:
                cookie_match = re.search(r'Cookie: ([^\'\"\\n]+)', fresh_curl)
                if cookie_match:
                    cookie = cookie_match.group(1)
                    auth_key = f"{platform_id}_COOKIE".upper()
                    print(f"🍪 Extracted cookie - có thể lưu: userdata.set('{auth_key}', 'your_cookie')")
            
            return fresh_curl
        else:
            print(f"❌ Không có fresh cURL cho {platform_name}")
            return None
    
    def build_authenticated_curl(self, platform_id, platform_name, base_template):
        """Build cURL với authentication từ userdata hoặc fresh input"""
        
        # Try to get stored auth first
        stored_auth = None
        for auth_type in ['TOKEN', 'COOKIE', 'API_KEY']:
            auth_key = f"{platform_id}_{auth_type}".upper()
            try:
                stored_value = userdata.get(auth_key)
                if stored_value:
                    stored_auth = {'type': auth_type, 'value': stored_value}
                    print(f"🔍 Found stored {auth_type} cho {platform_name}")
                    break
            except:
                continue
        
        if stored_auth:
            # Try to use stored auth
            use_stored = input(f"🔄 Sử dụng stored {stored_auth['type']} cho {platform_name}? (y/N): ").strip().lower()
            
            if use_stored == 'y':
                # Build cURL với stored auth
                auth_curl = self._inject_auth_into_curl(base_template, stored_auth)
                if auth_curl:
                    return auth_curl
        
        # Get fresh cURL if no stored auth hoặc user không muốn dùng
        return self.get_fresh_curl_with_auth(platform_id, platform_name, base_template)
    
    def _inject_auth_into_curl(self, base_template, auth_info):
        """Inject authentication vào base cURL template"""
        try:
            if auth_info['type'] == 'TOKEN':
                if 'Authorization:' not in base_template:
                    # Add authorization header
                    if '-H' in base_template:
                        base_template = base_template.replace(
                            '-H', 
                            f'-H "Authorization: Bearer {auth_info["value"]}" -H', 
                            1
                        )
                    else:
                        base_template = base_template.replace(
                            'curl ', 
                            f'curl -H "Authorization: Bearer {auth_info["value"]}" '
                        )
                else:
                    # Replace existing authorization
                    base_template = re.sub(
                        r'Authorization: Bearer [^\s\'"]+',
                        f'Authorization: Bearer {auth_info["value"]}',
                        base_template
                    )
            
            elif auth_info['type'] == 'COOKIE':
                if 'Cookie:' not in base_template:
                    # Add cookie header
                    if '-H' in base_template:
                        base_template = base_template.replace(
                            '-H', 
                            f'-H "Cookie: {auth_info["value"]}" -H', 
                            1
                        )
                    else:
                        base_template = base_template.replace(
                            'curl ', 
                            f'curl -H "Cookie: {auth_info["value"]}" '
                        )
                else:
                    # Replace existing cookie
                    base_template = re.sub(
                        r'Cookie: [^\n\'"]+',
                        f'Cookie: {auth_info["value"]}',
                        base_template
                    )
            
            print(f"✅ Injected {auth_info['type']} vào cURL template")
            return base_template
            
        except Exception as e:
            print(f"❌ Error injecting auth: {e}")
            return None

# Initialize secure platform auth manager
platform_auth_manager = SecurePlatformAuthManager(credential_manager)
print("🔐 Secure Platform Authentication Manager initialized!")


---
# 🔐 Secure Credentials Usage Guide

Cách sử dụng Google Colab userdata để lưu credentials an toàn


In [ ]:
# @title 💡 Secure Credentials Setup Guide
# @markdown Hướng dẫn setup credentials an toàn

def show_credentials_setup_guide():
    """Display guide cho việc setup secure credentials"""
    print("🔐 SECURE CREDENTIALS SETUP GUIDE")
    print("=" * 50)
    
    print("\n📋 REQUIRED CREDENTIALS:")
    print("-" * 30)
    print("🔑 GitHub Token:")
    print("   userdata.set('GITHUB_TOKEN', 'ghp_your_token_here')")
    print("   👆 Get from: https://github.com/settings/tokens")
    
    print("\n📂 GitHub Repository:")
    print("   userdata.set('GITHUB_REPO', 'username/repository')")
    print("   👆 Format: username/repo-name")
    
    print("\n🌿 Git Branch (optional):")
    print("   userdata.set('BRANCH_NAME', 'main')")
    print("   👆 Default: 'main'")
    
    print("\n📋 OPTIONAL CREDENTIALS:")
    print("-" * 30)
    print("☁️ Cloudinary (for image processing):")
    print("   userdata.set('CLOUDINARY_CLOUD_NAME', 'your_cloud')")
    print("   userdata.set('CLOUDINARY_API_KEY', 'your_key')")
    print("   userdata.set('CLOUDINARY_API_SECRET', 'your_secret')")
    
    print("\n🔗 Default Config API:")
    print("   userdata.set('DEFAULT_CONFIG_API', 'https://your-api.com')")
    
    print("\n📋 PLATFORM AUTHENTICATION:")
    print("-" * 30)
    print("🍪 Platform Cookies/Tokens (ví dụ cho YeuMoney):")
    print("   userdata.set('YEUMONEY_TOKEN', 'your_token')")
    print("   userdata.set('YEUMONEY_COOKIE', 'session=abc123;auth=xyz')")
    print("   userdata.set('SITE2S_TOKEN', 'your_token')")
    
    print("\n💡 USAGE TIPS:")
    print("-" * 20)
    print("✅ Credentials được lưu permanent trong session")
    print("✅ Không cần nhập lại khi restart notebook")
    print("✅ An toàn hơn so với hardcode trong code")
    print("✅ Automatically detect và sử dụng stored values")
    
    print("\n🛡️ SECURITY BEST PRACTICES:")
    print("-" * 30)
    print("🔒 Chỉ lưu credentials trong userdata khi cần")
    print("🔒 Không share notebook với credentials")
    print("🔒 Use minimal scope tokens cho GitHub")
    print("🔒 Revoke tokens khi không sử dụng")
    
    print("\n📋 QUICK SETUP COMMANDS:")
    print("-" * 25)
    print("# Copy và run những dòng sau để setup nhanh:")
    print("from google.colab import userdata")
    print("userdata.set('GITHUB_TOKEN', 'your_token')")
    print("userdata.set('GITHUB_REPO', 'username/repo')")
    print("# Sau đó restart notebook để load credentials")

# Run setup guide
show_credentials_setup_guide()


In [ ]:
# @title 📚 Google Drive & Epub Processing Support
# @markdown Support libraries cho Mode 2 & 3

# Additional imports for epub processing
try:
    import ebooklib
    from ebooklib import epub
    print("✅ ebooklib imported successfully")
except ImportError:
    print("⚠️ Installing ebooklib...")
    !pip install ebooklib -q
    import ebooklib
    from ebooklib import epub
    print("✅ ebooklib installed and imported")

# Additional imports for image processing
try:
    from PIL import Image
    import cloudinary
    import cloudinary.uploader
    import cloudinary.api
    print("✅ Image processing libraries available")
except ImportError:
    print("⚠️ Installing image processing libraries...")
    !pip install Pillow cloudinary -q
    from PIL import Image
    import cloudinary
    import cloudinary.uploader
    import cloudinary.api
    print("✅ Image processing libraries installed")

# Google Drive processing
import io
from urllib.parse import parse_qs, urlparse

class GoogleDriveProcessor:
    """Process Google Drive links và files"""
    
    def __init__(self):
        self.drive_mounted = False
        
    def mount_drive(self):
        """Mount Google Drive if not already mounted"""
        if not self.drive_mounted:
            try:
                drive.mount('/content/drive')
                self.drive_mounted = True
                print("✅ Google Drive mounted successfully")
                return True
            except Exception as e:
                print(f"❌ Failed to mount Google Drive: {e}")
                return False
        return True
    
    def extract_file_id_from_url(self, drive_url):
        """Extract file ID từ Google Drive URL"""
        try:
            if '/file/d/' in drive_url:
                # Standard sharing link
                file_id = drive_url.split('/file/d/')[1].split('/')[0]
                return file_id
            elif 'id=' in drive_url:
                # Query parameter format
                parsed = urlparse(drive_url)
                query_params = parse_qs(parsed.query)
                return query_params.get('id', [None])[0]
            else:
                print(f"❌ Unsupported Google Drive URL format: {drive_url}")
                return None
        except Exception as e:
            print(f"❌ Error extracting file ID: {e}")
            return None
    
    def get_direct_download_url(self, file_id):
        """Convert file ID to direct download URL"""
        return f"https://drive.google.com/uc?export=download&id={file_id}"
    
    def download_file_from_drive(self, drive_url, local_path):
        """Download file from Google Drive URL"""
        try:
            file_id = self.extract_file_id_from_url(drive_url)
            if not file_id:
                return False
            
            download_url = self.get_direct_download_url(file_id)
            
            # Download file
            response = requests.get(download_url)
            
            if response.status_code == 200:
                with open(local_path, 'wb') as f:
                    f.write(response.content)
                print(f"✅ Downloaded: {local_path}")
                return True
            else:
                print(f"❌ Download failed: HTTP {response.status_code}")
                return False
                
        except Exception as e:
            print(f"❌ Error downloading file: {e}")
            return False
    
    def create_public_sharing_link(self, file_path):
        """Create public sharing link for file in user's Drive"""
        try:
            if not self.mount_drive():
                return None
            
            # This would require Google Drive API setup
            # For now, we'll assume user provides the public link
            print("⚠️ Please create public sharing link manually and provide it")
            public_link = input("📂 Public sharing link: ").strip()
            return public_link if public_link else None
            
        except Exception as e:
            print(f"❌ Error creating public link: {e}")
            return None

class EpubProcessor:
    """Process EPUB files và extract metadata"""
    
    def __init__(self):
        self.temp_dir = "/tmp/epub_processing"
        os.makedirs(self.temp_dir, exist_ok=True)
    
    def extract_metadata(self, epub_path):
        """Extract metadata từ EPUB file"""
        try:
            book = epub.read_epub(epub_path)
            
            metadata = {
                'title': '',
                'author': '',
                'description': '',
                'language': 'Tiếng Việt',
                'publisher': '',
                'published_date': '',
                'isbn': '',
                'pages': 0,
                'preview_content': ''
            }
            
            # Extract basic metadata
            title = book.get_metadata('DC', 'title')
            if title:
                metadata['title'] = title[0][0] if title else 'Unknown Title'
            
            creator = book.get_metadata('DC', 'creator')
            if creator:
                metadata['author'] = creator[0][0] if creator else 'Unknown Author'
            
            description = book.get_metadata('DC', 'description')
            if description:
                metadata['description'] = description[0][0] if description else ''
            
            language = book.get_metadata('DC', 'language')
            if language:
                metadata['language'] = language[0][0] if language else 'Tiếng Việt'
            
            publisher = book.get_metadata('DC', 'publisher')
            if publisher:
                metadata['publisher'] = publisher[0][0] if publisher else ''
            
            date = book.get_metadata('DC', 'date')
            if date:
                metadata['published_date'] = date[0][0] if date else ''
            
            identifier = book.get_metadata('DC', 'identifier')
            if identifier:
                for ident in identifier:
                    if 'isbn' in ident[1].get('scheme', '').lower():
                        metadata['isbn'] = ident[0]
                        break
            
            print(f"✅ Extracted metadata for: {metadata['title']}")
            return metadata
            
        except Exception as e:
            print(f"❌ Error extracting metadata: {e}")
            return None
    
    def extract_cover_image(self, epub_path):
        """Extract cover image từ EPUB"""
        try:
            book = epub.read_epub(epub_path)
            
            # Try to find cover image
            cover_image = None
            
            # Method 1: Look for cover in metadata
            cover_meta = book.get_metadata('OPF', 'cover')
            if cover_meta:
                cover_id = cover_meta[0][0]
                for item in book.get_items():
                    if item.get_id() == cover_id:
                        cover_image = item.get_content()
                        break
            
            # Method 2: Look for common cover file names
            if not cover_image:
                cover_names = ['cover.jpg', 'cover.jpeg', 'cover.png', 'cover.gif']
                for item in book.get_items():
                    if item.get_type() == ebooklib.ITEM_IMAGE:
                        filename = item.get_name().lower()
                        if any(name in filename for name in cover_names):
                            cover_image = item.get_content()
                            break
            
            # Method 3: Take first image
            if not cover_image:
                for item in book.get_items():
                    if item.get_type() == ebooklib.ITEM_IMAGE:
                        cover_image = item.get_content()
                        break
            
            if cover_image:
                cover_path = os.path.join(self.temp_dir, "cover.jpg")
                with open(cover_path, 'wb') as f:
                    f.write(cover_image)
                print(f"✅ Extracted cover image: {cover_path}")
                return cover_path
            else:
                print("⚠️ No cover image found in EPUB")
                return None
                
        except Exception as e:
            print(f"❌ Error extracting cover: {e}")
            return None
    
    def extract_preview_content(self, epub_path, max_chars=500):
        """Extract preview text từ EPUB"""
        try:
            book = epub.read_epub(epub_path)
            
            preview_text = ""
            
            # Get readable items (chapters)
            for item in book.get_items():
                if item.get_type() == ebooklib.ITEM_DOCUMENT:
                    content = item.get_content().decode('utf-8')
                    
                    # Simple HTML tag removal
                    import re
                    text = re.sub(r'<[^>]+>', '', content)
                    text = re.sub(r'\s+', ' ', text).strip()
                    
                    if text and len(text) > 50:  # Skip short/empty chapters
                        preview_text = text[:max_chars]
                        break
            
            if preview_text:
                print(f"✅ Extracted preview ({len(preview_text)} chars)")
                return preview_text
            else:
                print("⚠️ No readable content found for preview")
                return ""
                
        except Exception as e:
            print(f"❌ Error extracting preview: {e}")
            return ""

# Initialize processors
drive_processor = GoogleDriveProcessor()
epub_processor = EpubProcessor()

print("📚 Google Drive & Epub Processing initialized!")
print("🔧 Ready for Mode 2 & 3 operations")


In [ ]:
# @title ☁️ Cloudinary Image Manager
# @markdown Process và upload images to Cloudinary với optimal settings

class CloudinaryImageManager:
    """Manage image processing và upload với Cloudinary"""
    
    def __init__(self, credential_manager):
        self.credential_manager = credential_manager
        self.configured = False
        self.setup_cloudinary()
    
    def setup_cloudinary(self):
        """Setup Cloudinary configuration"""
        try:
            cloud_name = self.credential_manager.get('cloudinary_cloud_name')
            api_key = self.credential_manager.get('cloudinary_api_key')
            api_secret = self.credential_manager.get('cloudinary_api_secret')
            
            if cloud_name and api_key and api_secret:
                cloudinary.config(
                    cloud_name=cloud_name,
                    api_key=api_key,
                    api_secret=api_secret
                )
                self.configured = True
                print(f"✅ Cloudinary configured: {cloud_name}")
            else:
                print("⚠️ Cloudinary not configured - image processing disabled")
                self.configured = False
                
        except Exception as e:
            print(f"❌ Cloudinary setup failed: {e}")
            self.configured = False
    
    def optimize_image(self, image_path, target_width, target_height, quality=80):
        """Optimize image dimensions và quality"""
        try:
            with Image.open(image_path) as img:
                # Convert to RGB if needed
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                
                # Calculate resize dimensions maintaining aspect ratio
                img_ratio = img.width / img.height
                target_ratio = target_width / target_height
                
                if img_ratio > target_ratio:
                    # Image is wider, fit by height
                    new_height = target_height
                    new_width = int(target_height * img_ratio)
                else:
                    # Image is taller, fit by width
                    new_width = target_width
                    new_height = int(target_width / img_ratio)
                
                # Resize image
                resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                
                # Save optimized version
                optimized_path = image_path.replace('.jpg', '_optimized.jpg')
                resized_img.save(optimized_path, 'JPEG', quality=quality, optimize=True)
                
                print(f"✅ Optimized: {img.width}x{img.height} → {new_width}x{new_height}")
                return optimized_path
                
        except Exception as e:
            print(f"❌ Image optimization failed: {e}")
            return image_path
    
    def upload_cover_image(self, image_path):
        """Upload cover image với optimal settings cho epub cards"""
        if not self.configured:
            print("⚠️ Cloudinary not configured, returning local path")
            return image_path
        
        try:
            # Optimize for cover display (400x600px, WebP 80% quality)
            optimized_path = self.optimize_image(image_path, 400, 600, 80)
            
            # Upload to Cloudinary
            upload_result = cloudinary.uploader.upload(
                optimized_path,
                folder="epub_covers",
                format="webp",
                quality="auto:good",
                fetch_format="auto",
                responsive=True,
                width=400,
                height=600,
                crop="fit"
            )
            
            cloudinary_url = upload_result['secure_url']
            print(f"✅ Cover uploaded: {cloudinary_url}")
            
            # Cleanup local files
            if os.path.exists(optimized_path):
                os.remove(optimized_path)
            
            return cloudinary_url
            
        except Exception as e:
            print(f"❌ Cover upload failed: {e}")
            return image_path
    
    def upload_preview_image(self, image_path):
        """Upload preview image với larger dimensions"""
        if not self.configured:
            print("⚠️ Cloudinary not configured, returning local path")
            return image_path
        
        try:
            # Optimize for preview display (800x1200px, WebP 80% quality)
            optimized_path = self.optimize_image(image_path, 800, 1200, 80)
            
            # Upload to Cloudinary
            upload_result = cloudinary.uploader.upload(
                optimized_path,
                folder="epub_previews",
                format="webp",
                quality="auto:good",
                fetch_format="auto",
                responsive=True,
                width=800,
                height=1200,
                crop="fit"
            )
            
            cloudinary_url = upload_result['secure_url']
            print(f"✅ Preview uploaded: {cloudinary_url}")
            
            # Cleanup local files
            if os.path.exists(optimized_path):
                os.remove(optimized_path)
            
            return cloudinary_url
            
        except Exception as e:
            print(f"❌ Preview upload failed: {e}")
            return image_path
    
    def process_epub_images(self, cover_path, preview_path=None):
        """Process both cover và preview images"""
        results = {}
        
        if cover_path and os.path.exists(cover_path):
            results['cover_url'] = self.upload_cover_image(cover_path)
        else:
            results['cover_url'] = "https://via.placeholder.com/400x600/cccccc/ffffff?text=No+Cover"
        
        if preview_path and os.path.exists(preview_path):
            results['preview_url'] = self.upload_preview_image(preview_path)
        else:
            results['preview_url'] = results['cover_url']  # Use cover as preview fallback
        
        return results

# Initialize cloudinary manager
if credential_manager:
    cloudinary_manager = CloudinaryImageManager(credential_manager)
    print("☁️ Cloudinary Image Manager initialized!")
else:
    cloudinary_manager = None
    print("⚠️ Cloudinary Image Manager not initialized - missing credential manager")


---
# 📂 Mode 2: Google Drive Epub Extraction

Extract thông tin từ single Google Drive EPUB file


In [ ]:
# @title 📂 Mode 2: Google Drive Epub Extraction
# @markdown Extract metadata từ single Google Drive EPUB link

def mode2_gdrive_epub_extraction():
    """Mode 2: Extract book từ Google Drive EPUB link"""
    
    if not github_manager or not platform_config_manager:
        print("❌ Required managers not available!")
        print("🔧 Please configure GitHub credentials first")
        return
    
    print("📂 Mode 2: Google Drive Epub Extraction")
    print("=" * 50)
    
    # Get Google Drive link
    print("📋 Google Drive Information:")
    drive_url = input("🔗 Google Drive EPUB URL: ").strip()
    if not drive_url:
        print("❌ Google Drive URL is required!")
        return
    
    # Validate Google Drive URL
    if 'drive.google.com' not in drive_url:
        print("❌ Invalid Google Drive URL!")
        return
    
    file_id = drive_processor.extract_file_id_from_url(drive_url)
    if not file_id:
        print("❌ Could not extract file ID from URL!")
        return
    
    print(f"✅ Extracted file ID: {file_id}")
    
    # Download EPUB file
    temp_epub_path = f"/tmp/epub_{file_id}.epub"
    print(f"\n📥 Downloading EPUB file...")
    
    if not drive_processor.download_file_from_drive(drive_url, temp_epub_path):
        print("❌ Failed to download EPUB file!")
        return
    
    # Extract metadata
    print(f"\n📖 Extracting metadata...")
    metadata = epub_processor.extract_metadata(temp_epub_path)
    if not metadata:
        print("❌ Failed to extract metadata!")
        return
    
    # Check for duplicates
    print(f"\n🔍 Checking for duplicates...")
    is_duplicate, existing_file = github_manager.check_duplicate_book(
        metadata['title'], metadata['author']
    )
    
    if is_duplicate:
        print(f"⚠️ Book already exists: {existing_file}")
        overwrite = input("📝 Continue anyway? (y/N): ").strip().lower()
        if overwrite != 'y':
            print("❌ Operation cancelled")
            # Cleanup
            if os.path.exists(temp_epub_path):
                os.remove(temp_epub_path)
            return
    
    # Extract images
    print(f"\n🖼️ Processing images...")
    cover_path = epub_processor.extract_cover_image(temp_epub_path)
    
    # Process với Cloudinary nếu available
    image_urls = {}
    if cloudinary_manager and cloudinary_manager.configured:
        print("☁️ Uploading images to Cloudinary...")
        image_urls = cloudinary_manager.process_epub_images(cover_path)
    else:
        print("⚠️ Cloudinary not configured, using placeholder images")
        image_urls = {
            'cover_url': "https://via.placeholder.com/400x600/cccccc/ffffff?text=No+Cover",
            'preview_url': "https://via.placeholder.com/800x1200/cccccc/ffffff?text=No+Preview"
        }
    
    # Extract preview content
    print(f"\n📄 Extracting preview content...")
    preview_content = epub_processor.extract_preview_content(temp_epub_path)
    
    # Review extracted information
    print(f"\n📋 Extracted Information:")
    print("=" * 40)
    print(f"📖 Title: {metadata['title']}")
    print(f"✍️ Author: {metadata['author']}")
    print(f"📄 Description: {metadata['description'][:100]}..." if metadata['description'] else "📄 Description: (empty)")
    print(f"🌐 Language: {metadata['language']}")
    print(f"🏢 Publisher: {metadata['publisher']}")
    print(f"📅 Published: {metadata['published_date']}")
    print(f"📊 ISBN: {metadata['isbn']}")
    print(f"🖼️ Cover: {image_urls['cover_url'][:50]}...")
    print(f"📄 Preview: {len(preview_content)} characters")
    
    # Allow editing
    print(f"\n✏️ Review and Edit Information:")
    edit_confirm = input("📝 Edit any information? (y/N): ").strip().lower()
    
    if edit_confirm == 'y':
        print("\n📝 Edit Mode (press Enter to keep current value):")
        
        new_title = input(f"📖 Title [{metadata['title']}]: ").strip()
        if new_title:
            metadata['title'] = new_title
        
        new_author = input(f"✍️ Author [{metadata['author']}]: ").strip()
        if new_author:
            metadata['author'] = new_author
        
        new_description = input(f"📄 Description [{metadata['description'][:50]}...]: ").strip()
        if new_description:
            metadata['description'] = new_description
        
        new_language = input(f"🌐 Language [{metadata['language']}]: ").strip()
        if new_language:
            metadata['language'] = new_language
        
        # Genre input
        genres_input = input("🏷️ Genres (comma separated): ").strip()
        genres = [g.strip() for g in genres_input.split(',')] if genres_input else []
        
        # Rating and pages
        rating_input = input("⭐ Rating (1-5, optional): ").strip()
        rating = float(rating_input) if rating_input and rating_input.replace('.', '').isdigit() else None
    else:
        genres = []
        rating = None
    
    # Platform shortening setup
    print(f"\n🔗 Platform Shortening Setup:")
    print("Available platforms:")
    if platform_config_manager:
        platform_config_manager.list_platforms()
    
    # Use the original Google Drive sharing URL for shortening
    download_links = []
    shortener_platforms = [p for p in platform_config_manager.platforms.values() if p['type'] == 'shortener']
    
    if shortener_platforms:
        print(f"\n🚀 Auto-shortening với all available platforms...")
        
        for platform_info in shortener_platforms:
            print(f"\n🔧 Processing {platform_info['name']}...")
            
            # Get authenticated cURL (userdata or fresh input)
            authenticated_curl = platform_auth_manager.build_authenticated_curl(
                platform_info['id'], 
                platform_info['name'], 
                platform_info.get('curl_template', '')
            )
            
            if authenticated_curl:
                # Update platform với authenticated cURL
                dynamic_platform_manager.add_platform_from_curl(
                    platform_info['name'], 
                    authenticated_curl, 
                    platform_info.get('response_config')
                )
                
                # Shorten the original Google Drive URL
                shortened_url = dynamic_platform_manager.shorten_url(platform_info['name'], drive_url)
                
                if shortened_url != drive_url:
                    download_links.append({
                        'platform': platform_info['name'],
                        'url': shortened_url,
                        'index': platform_info['index'],
                        'icon': platform_info['icon']
                    })
                    print(f"✅ {platform_info['name']}: {drive_url[:30]}... → {shortened_url}")
                else:
                    print(f"⚠️ {platform_info['name']}: Shortening failed, skipping")
            else:
                print(f"⚠️ {platform_info['name']}: No authentication, skipping")
    
    # Add direct Google Drive link as fallback
    gdrive_platform = platform_config_manager.get_platform_by_index(0)  # Assuming Google Drive is index 0
    if gdrive_platform:
        download_links.append({
            'platform': gdrive_platform['name'],
            'url': drive_url,
            'index': gdrive_platform['index'],
            'icon': gdrive_platform['icon']
        })
        print(f"✅ Added direct Google Drive link")
    
    if not download_links:
        print("❌ No download links available!")
        return
    
    # Build book data
    book_data = {
        'title': metadata['title'],
        'author': metadata['author'],
        'description': metadata['description'],
        'cover_image': image_urls['cover_url'],
        'preview_image': image_urls['preview_url'],
        'language': metadata['language'],
        'download_links': download_links,
        'config_url': credential_manager.get('default_config_api') or ""
    }
    
    # Add optional fields
    if metadata.get('publisher'):\n        book_data['publisher'] = metadata['publisher']\n    if metadata.get('published_date'):\n        book_data['published_date'] = metadata['published_date']\n    if metadata.get('isbn'):\n        book_data['isbn'] = metadata['isbn']\n    if genres:\n        book_data['genre'] = genres\n    if rating:\n        book_data['rating'] = rating\n    if preview_content:\n        book_data['preview_content'] = preview_content\n    \n    # Generate markdown\n    print(f\"\\n📝 Generating markdown file...\")\n    markdown_content = markdown_generator.generate_markdown(book_data)\n    filename = markdown_generator.generate_filename(metadata['title'])\n    \n    print(f\"📄 Generated file: {filename}\")\n    print(f\"📏 Content length: {len(markdown_content)} characters\")\n    print(f\"🔗 Download links: {len(download_links)}\")\n    \n    # Preview\n    print(f\"\\n📋 Preview (first 300 chars):\")\n    print(\"-\" * 40)\n    print(markdown_content[:300] + \"...\")\n    print(\"-\" * 40)\n    \n    # Confirm upload\n    upload_confirm = input(f\"\\n📤 Upload '{filename}' to GitHub? (y/N): \").strip().lower()\n    \n    if upload_confirm == 'y':\n        file_path = f\"_epubs/{filename}\"\n        commit_message = f\"Thêm sách từ Google Drive: {metadata['title']} - {metadata['author']}\"\n        \n        try:\n            if github_manager.file_exists(file_path):\n                success = github_manager.update_file(file_path, markdown_content, commit_message)\n                action = \"Updated\"\n            else:\n                success = github_manager.create_file(file_path, markdown_content, commit_message)\n                action = \"Created\"\n            \n            if success:\n                print(f\"✅ {action} file successfully!\")\n                print(f\"📂 File: {file_path}\")\n                print(f\"💬 Commit: {commit_message}\")\n                print(f\"🚀 GitHub Pages sẽ rebuild automatically\")\n                \n                # Summary\n                print(f\"\\n📊 Summary:\")\n                print(f\"   📖 Title: {metadata['title']}\")\n                print(f\"   ✍️ Author: {metadata['author']}\")\n                print(f\"   📂 Source: Google Drive\")\n                print(f\"   🔗 Download links: {len(download_links)}\")\n                print(f\"   🖼️ Images: Processed\")\n                for link in download_links:\n                    print(f\"      [{link['index']}] {link['platform']}\")\n            else:\n                print(f\"❌ Failed to {action.lower()} file!\")\n                \n        except Exception as e:\n            print(f\"❌ Error uploading file: {e}\")\n    else:\n        print(\"❌ File not uploaded\")\n        print(f\"💾 Markdown content available for reference\")\n    \n    # Cleanup temporary files\n    print(f\"\\n🧹 Cleaning up temporary files...\")\n    if os.path.exists(temp_epub_path):\n        os.remove(temp_epub_path)\n    if cover_path and os.path.exists(cover_path):\n        os.remove(cover_path)\n    print(\"✅ Cleanup completed\")\n\n# Run Mode 2\nmode2_gdrive_epub_extraction()"


---
# 📁 Mode 3: Bulk Folder Processing

Xử lý bulk từ thư mục Google Drive chứa nhiều EPUB files


In [ ]:
# @title 📁 Mode 3: Bulk Folder Processing
# @markdown Xử lý bulk multiple EPUB files từ Google Drive folder

def mode3_bulk_folder_processing():
    """Mode 3: Bulk process EPUB files từ Google Drive folder"""
    
    if not github_manager or not platform_config_manager:
        print("❌ Required managers not available!")
        print("🔧 Please configure GitHub credentials first")
        return
    
    print("📁 Mode 3: Bulk Folder Processing")
    print("=" * 50)
    
    # Mount Google Drive
    print("📂 Mounting Google Drive...")
    if not drive_processor.mount_drive():
        print("❌ Failed to mount Google Drive!")
        return
    
    # Get folder path
    print("\n📋 Folder Information:")
    print("💡 Tip: Browse to your EPUB folder trong file browser và copy path")
    print("📝 Example: /content/drive/MyDrive/Books/EPUBs/")
    
    folder_path = input("📁 Folder path containing EPUB files: ").strip()
    if not folder_path:
        print("❌ Folder path is required!")
        return
    
    # Validate folder exists
    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return
    
    # Find EPUB files
    print(f"\n🔍 Scanning for EPUB files...")
    epub_files = []
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.epub'):
                epub_path = os.path.join(root, file)
                epub_files.append(epub_path)
    
    if not epub_files:
        print(f"❌ No EPUB files found in {folder_path}")
        return
    
    print(f"✅ Found {len(epub_files)} EPUB files:")
    for i, epub_path in enumerate(epub_files, 1):
        filename = os.path.basename(epub_path)
        print(f"   {i}. {filename}")
    
    # Confirm processing
    process_confirm = input(f"\n📋 Process all {len(epub_files)} EPUB files? (y/N): ").strip().lower()
    
    if process_confirm != 'y':
        print("❌ Bulk processing cancelled")
        return
    
    # Setup processing parameters
    print(f"\n⚙️ Processing Configuration:")
    
    # Batch size
    batch_size_input = input("📦 Batch size (default: 5): ").strip()
    batch_size = int(batch_size_input) if batch_size_input.isdigit() else 5
    
    # Skip duplicates option
    skip_duplicates = input("🔄 Skip duplicate books? (Y/n): ").strip().lower()
    skip_duplicates = skip_duplicates != 'n'
    
    # Get shortener platforms for bulk processing
    shortener_platforms = [p for p in platform_config_manager.platforms.values() if p['type'] == 'shortener']
    
    if shortener_platforms:
        print(f"\n🔗 Available shortener platforms: {len(shortener_platforms)}")
        for platform in shortener_platforms:
            print(f"   • {platform['name']} (index {platform['index']})")
        
        # Get fresh authentication for all platforms upfront
        platform_auths = {}
        for platform_info in shortener_platforms:
            print(f"\n🔐 Setting up authentication for {platform_info['name']}...")
            
            authenticated_curl = platform_auth_manager.build_authenticated_curl(
                platform_info['id'], 
                platform_info['name'], 
                platform_info.get('curl_template', '')
            )
            
            if authenticated_curl:
                # Setup platform for bulk processing
                dynamic_platform_manager.add_platform_from_curl(
                    platform_info['name'], 
                    authenticated_curl, 
                    platform_info.get('response_config')
                )
                platform_auths[platform_info['id']] = True
                print(f"✅ {platform_info['name']} ready for bulk processing")
            else:
                platform_auths[platform_info['id']] = False
                print(f"⚠️ {platform_info['name']} authentication failed, will skip")
    
    # Start bulk processing
    print(f"\n🚀 Starting bulk processing...")
    print(f"📦 Batch size: {batch_size}")
    print(f"🔄 Skip duplicates: {'Yes' if skip_duplicates else 'No'}")
    print("=" * 50)
    
    processed_count = 0
    success_count = 0
    duplicate_count = 0
    error_count = 0
    
    # Process in batches
    for batch_start in range(0, len(epub_files), batch_size):
        batch_end = min(batch_start + batch_size, len(epub_files))
        batch_files = epub_files[batch_start:batch_end]
        
        print(f"\n📦 Processing batch {batch_start//batch_size + 1}/{(len(epub_files)-1)//batch_size + 1}")
        print(f"📂 Files {batch_start + 1}-{batch_end} of {len(epub_files)}")
        
        for epub_path in batch_files:
            processed_count += 1
            filename = os.path.basename(epub_path)
            
            print(f"\n📖 [{processed_count}/{len(epub_files)}] Processing: {filename}")
            
            try:
                # Extract metadata
                print("   📝 Extracting metadata...")
                metadata = epub_processor.extract_metadata(epub_path)
                if not metadata:
                    print("   ❌ Failed to extract metadata, skipping")
                    error_count += 1
                    continue
                
                # Check for duplicates
                if skip_duplicates:
                    is_duplicate, existing_file = github_manager.check_duplicate_book(
                        metadata['title'], metadata['author']
                    )
                    
                    if is_duplicate:
                        print(f"   ⚠️ Duplicate found: {existing_file}, skipping")
                        duplicate_count += 1
                        continue
                
                # Create public sharing link
                print("   🔗 Creating public sharing link...")
                # For bulk processing, we'll create a simple file://path as placeholder
                # In real scenario, this would involve Google Drive API
                public_drive_link = f"https://drive.google.com/file/d/PLACEHOLDER_{processed_count}/view?usp=sharing"
                print(f"   📋 Note: Please manually create sharing link for {filename}")
                
                # Extract images
                print("   🖼️ Processing images...")
                cover_path = epub_processor.extract_cover_image(epub_path)
                
                # Process images
                image_urls = {}
                if cloudinary_manager and cloudinary_manager.configured:
                    print("   ☁️ Uploading to Cloudinary...")
                    image_urls = cloudinary_manager.process_epub_images(cover_path)
                else:
                    image_urls = {
                        'cover_url': f"https://via.placeholder.com/400x600/cccccc/ffffff?text={metadata['title'][:20]}",
                        'preview_url': f"https://via.placeholder.com/800x1200/cccccc/ffffff?text={metadata['title'][:20]}"
                    }
                
                # Platform shortening
                print("   🔗 Shortening URLs...")
                download_links = []
                
                # Process with available shortener platforms
                for platform_info in shortener_platforms:
                    if platform_auths.get(platform_info['id'], False):
                        shortened_url = dynamic_platform_manager.shorten_url(
                            platform_info['name'], 
                            public_drive_link
                        )
                        
                        if shortened_url != public_drive_link:
                            download_links.append({
                                'platform': platform_info['name'],
                                'url': shortened_url,
                                'index': platform_info['index'],
                                'icon': platform_info['icon']
                            })
                        
                        # Add delay để tránh rate limiting
                        time.sleep(1)
                
                # Add direct link
                gdrive_platform = platform_config_manager.get_platform_by_index(0)
                if gdrive_platform:
                    download_links.append({
                        'platform': gdrive_platform['name'],
                        'url': public_drive_link,
                        'index': gdrive_platform['index'],
                        'icon': gdrive_platform['icon']
                    })
                
                if not download_links:
                    print("   ⚠️ No download links available, skipping")
                    error_count += 1
                    continue
                
                # Build book data
                preview_content = epub_processor.extract_preview_content(epub_path)
                
                book_data = {
                    'title': metadata['title'],
                    'author': metadata['author'],
                    'description': metadata['description'],
                    'cover_image': image_urls['cover_url'],
                    'preview_image': image_urls['preview_url'],
                    'language': metadata['language'],
                    'download_links': download_links,
                    'config_url': credential_manager.get('default_config_api') or ""
                }
                
                # Add optional fields
                if metadata.get('publisher'):
                    book_data['publisher'] = metadata['publisher']
                if metadata.get('published_date'):
                    book_data['published_date'] = metadata['published_date']
                if metadata.get('isbn'):
                    book_data['isbn'] = metadata['isbn']
                if preview_content:
                    book_data['preview_content'] = preview_content
                
                # Generate và upload markdown
                print("   📝 Generating markdown...")
                markdown_content = markdown_generator.generate_markdown(book_data)
                md_filename = markdown_generator.generate_filename(metadata['title'])
                
                file_path = f"_epubs/{md_filename}"
                commit_message = f"Bulk add: {metadata['title']} - {metadata['author']}"
                
                # Upload to GitHub
                print("   📤 Uploading to GitHub...")
                if github_manager.file_exists(file_path):
                    success = github_manager.update_file(file_path, markdown_content, commit_message)
                else:
                    success = github_manager.create_file(file_path, markdown_content, commit_message)
                
                if success:
                    success_count += 1
                    print(f"   ✅ Success: {metadata['title']}")
                    print(f"      📂 File: {md_filename}")
                    print(f"      🔗 Links: {len(download_links)}")
                else:
                    error_count += 1
                    print(f"   ❌ Upload failed: {metadata['title']}")
                
                # Cleanup
                if cover_path and os.path.exists(cover_path):
                    os.remove(cover_path)
                
            except Exception as e:
                error_count += 1
                print(f"   ❌ Error processing {filename}: {e}")
                continue
        
        # Batch completion message
        print(f"\n📊 Batch {batch_start//batch_size + 1} completed")
        print(f"   ✅ Success: {success_count}")
        print(f"   ⚠️ Duplicates: {duplicate_count}")
        print(f"   ❌ Errors: {error_count}")
        
        # Pause between batches (nếu không phải batch cuối)
        if batch_end < len(epub_files):
            delay = input(f"\n⏸️ Pause before next batch? Enter seconds (default: 3): ").strip()
            delay_time = int(delay) if delay.isdigit() else 3
            print(f"⏱️ Waiting {delay_time} seconds...")
            time.sleep(delay_time)
    
    # Final summary
    print(f"\n🎉 BULK PROCESSING COMPLETED!")
    print("=" * 50)
    print(f"📂 Total files processed: {processed_count}")
    print(f"✅ Successfully added: {success_count}")
    print(f"⚠️ Duplicates skipped: {duplicate_count}")
    print(f"❌ Errors encountered: {error_count}")
    print(f"📈 Success rate: {(success_count/processed_count)*100:.1f}%")
    
    if success_count > 0:
        print(f"\n🚀 GitHub Pages will rebuild automatically")
        print(f"📚 {success_count} new books added to library")
        
        # Show processing stats
        print(f"\n📊 Processing Statistics:")
        if shortener_platforms:
            print(f"   🔗 Platform shortening: {len(shortener_platforms)} platforms used")
        if cloudinary_manager and cloudinary_manager.configured:
            print(f"   ☁️ Image processing: Cloudinary enabled")
        print(f"   📦 Batch size: {batch_size}")
        print(f"   ⏱️ Total time: ~{processed_count * 2} seconds estimated")
    
    print(f"\n💡 Notes:")
    print(f"   📝 Remember to manually create public sharing links")
    print(f"   🔗 Replace PLACEHOLDER links with real Google Drive URLs")
    print(f"   🔄 Re-run shortening for updated links if needed")

# Run Mode 3
mode3_bulk_folder_processing()


---
# 🔄 Legacy Book Conversion Tool

Convert existing books khi có platform mới


In [ ]:
# @title 🔄 Legacy Book Conversion Tool
# @markdown Convert existing books với platform mới

def legacy_book_conversion():
    """Convert tất cả sách cũ với platform mới"""
    
    if not github_manager or not platform_config_manager:
        print("❌ Required managers not available!")
        print("🔧 Please configure GitHub credentials first")
        return
    
    print("🔄 Legacy Book Conversion Tool")
    print("=" * 50)
    
    # Get list of all existing epub files
    print("📂 Scanning existing books...")
    
    try:
        epub_contents = github_manager.repo.get_contents("_epubs", ref=github_manager.branch)
        existing_books = [content for content in epub_contents if content.name.endswith('.md')]
        
        if not existing_books:
            print("❌ No existing books found!")
            return
        
        print(f"✅ Found {len(existing_books)} existing books")
        
    except Exception as e:
        print(f"❌ Error scanning books: {e}")
        return
    
    # Show available platforms
    print(f"\n🔗 Available Platforms:")
    platform_config_manager.list_platforms()
    
    # Get shortener platforms
    shortener_platforms = [p for p in platform_config_manager.platforms.values() if p['type'] == 'shortener']
    
    if not shortener_platforms:
        print("❌ No shortener platforms available for conversion!")
        return
    
    # Select platform for conversion
    print(f"\n🎯 Platform Selection:")
    print("Available shortener platforms:")
    for i, platform in enumerate(shortener_platforms, 1):
        print(f"   {i}. {platform['name']} (index {platform['index']})")
    
    platform_choice = input(f"\nSelect platform number (1-{len(shortener_platforms)}): ").strip()
    
    try:
        platform_index = int(platform_choice) - 1
        if 0 <= platform_index < len(shortener_platforms):
            selected_platform = shortener_platforms[platform_index]
        else:
            print("❌ Invalid platform selection!")
            return
    except ValueError:
        print("❌ Invalid input!")
        return
    
    print(f"✅ Selected: {selected_platform['name']}")
    
    # Get authentication for selected platform
    print(f"\n🔐 Setting up authentication for {selected_platform['name']}...")
    
    authenticated_curl = platform_auth_manager.build_authenticated_curl(
        selected_platform['id'], 
        selected_platform['name'], 
        selected_platform.get('curl_template', '')
    )
    
    if not authenticated_curl:
        print(f"❌ Authentication failed for {selected_platform['name']}!")
        return
    
    # Setup platform for processing
    dynamic_platform_manager.add_platform_from_curl(
        selected_platform['name'], 
        authenticated_curl, 
        selected_platform.get('response_config')
    )
    
    print(f"✅ Platform {selected_platform['name']} ready for conversion")
    
    # Processing options
    print(f"\n⚙️ Conversion Options:")
    
    # Batch size
    batch_size_input = input("📦 Batch size (default: 10): ").strip()
    batch_size = int(batch_size_input) if batch_size_input.isdigit() else 10
    
    # Test mode
    test_mode = input("🧪 Test mode (preview changes without committing)? (y/N): ").strip().lower() == 'y'
    
    # Confirmation
    if not test_mode:
        confirm = input(f"\n⚠️  CONFIRM: Convert {len(existing_books)} books với {selected_platform['name']}? (type 'CONVERT'): ").strip()
        if confirm != 'CONVERT':
            print("❌ Conversion cancelled")
            return
    
    # Start conversion
    print(f"\n🚀 Starting legacy book conversion...")
    print(f"📦 Batch size: {batch_size}")
    print(f"🧪 Test mode: {'Yes' if test_mode else 'No'}")
    print("=" * 50)
    
    processed_count = 0
    success_count = 0
    error_count = 0
    no_gdrive_count = 0
    
    # Process in batches
    for batch_start in range(0, len(existing_books), batch_size):
        batch_end = min(batch_start + batch_size, len(existing_books))
        batch_books = existing_books[batch_start:batch_end]
        
        print(f"\n📦 Processing batch {batch_start//batch_size + 1}/{(len(existing_books)-1)//batch_size + 1}")
        print(f"📂 Books {batch_start + 1}-{batch_end} of {len(existing_books)}")
        
        for book_content in batch_books:
            processed_count += 1
            book_filename = book_content.name
            
            print(f"\n📖 [{processed_count}/{len(existing_books)}] Processing: {book_filename}")
            
            try:
                # Get book content
                print("   📥 Reading book content...")
                file_content = github_manager.get_file_content(f"_epubs/{book_filename}")
                if not file_content:
                    print("   ❌ Failed to read book content")
                    error_count += 1
                    continue
                
                # Parse YAML front matter
                print("   📝 Parsing metadata...")
                if not file_content.startswith('---'):
                    print("   ❌ Invalid file format (no YAML front matter)")
                    error_count += 1
                    continue
                
                # Split content into YAML and body
                parts = file_content.split('---')
                if len(parts) < 3:
                    print("   ❌ Invalid YAML format")
                    error_count += 1
                    continue
                
                yaml_content = parts[1]
                body_content = '---'.join(parts[2:])
                
                # Parse YAML
                try:
                    import yaml
                    book_data = yaml.safe_load(yaml_content)
                except Exception as e:
                    print(f"   ❌ YAML parsing error: {e}")
                    error_count += 1
                    continue
                
                # Extract existing download links
                existing_links = book_data.get('download_links', [])
                if not existing_links:
                    print("   ⚠️ No download links found")
                    no_gdrive_count += 1
                    continue
                
                # Find Google Drive links
                gdrive_links = []
                for link in existing_links:
                    if ('drive.google.com' in link.get('url', '') and 
                        link.get('platform', '').lower() in ['google drive', 'gdrive']):
                        gdrive_links.append(link['url'])
                
                if not gdrive_links:
                    print("   ⚠️ No Google Drive links found to convert")
                    no_gdrive_count += 1
                    continue
                
                print(f"   🔍 Found {len(gdrive_links)} Google Drive links")
                
                # Process shortening for new platform
                new_links_added = []
                
                for gdrive_url in gdrive_links:
                    # Decode URL if obfuscated
                    actual_url = gdrive_url
                    if gdrive_url.startswith('data:encoded,'):
                        decoded_url = link_obfuscator.decode(gdrive_url)
                        if decoded_url:
                            actual_url = decoded_url
                    
                    print(f"   🔗 Shortening: {actual_url[:50]}...")
                    
                    # Shorten with new platform
                    shortened_url = dynamic_platform_manager.shorten_url(
                        selected_platform['name'], 
                        actual_url
                    )
                    
                    if shortened_url != actual_url:
                        new_link = {
                            'platform': selected_platform['name'],
                            'url': shortened_url,
                            'index': selected_platform['index'],
                            'icon': selected_platform['icon']
                        }
                        new_links_added.append(new_link)
                        print(f"   ✅ Shortened: {shortened_url}")
                    else:
                        print(f"   ⚠️ Shortening failed")
                    
                    # Delay để tránh rate limiting
                    time.sleep(1)
                
                if not new_links_added:
                    print("   ❌ No new links generated")
                    error_count += 1
                    continue
                
                # Check if platform already exists trong book
                platform_exists = any(
                    link.get('index') == selected_platform['index'] 
                    for link in existing_links
                )
                
                if platform_exists:
                    print(f"   ⚠️ Platform {selected_platform['name']} already exists, updating...")
                    # Update existing platform link
                    for i, link in enumerate(existing_links):
                        if link.get('index') == selected_platform['index']:
                            existing_links[i] = new_links_added[0]  # Use first new link
                            break
                else:
                    print(f"   ✅ Adding new platform {selected_platform['name']}")
                    # Add new platform links
                    existing_links.extend(new_links_added)
                
                # Update book data
                book_data['download_links'] = existing_links
                
                # Generate updated YAML content
                updated_yaml = yaml.dump(book_data, default_flow_style=False, allow_unicode=True)
                updated_content = f"---\n{updated_yaml}---{body_content}"
                
                if test_mode:
                    print("   🧪 TEST MODE: Changes prepared (not committed)")
                    print(f"      📊 New links: {len(new_links_added)}")
                    print(f"      📏 Content length: {len(updated_content)} chars")
                    success_count += 1
                else:
                    # Commit changes
                    print("   📤 Updating book...")
                    commit_message = f"Add {selected_platform['name']} links: {book_data.get('title', 'Unknown')}"
                    
                    success = github_manager.update_file(
                        f"_epubs/{book_filename}",
                        updated_content,
                        commit_message
                    )
                    
                    if success:
                        success_count += 1
                        print(f"   ✅ Updated: {book_data.get('title', book_filename)}")
                        print(f"      📊 Added {len(new_links_added)} new links")
                    else:
                        error_count += 1
                        print(f"   ❌ Update failed")
                
            except Exception as e:
                error_count += 1
                print(f"   ❌ Error processing {book_filename}: {e}")
                continue
        
        # Batch completion message
        print(f"\n📊 Batch {batch_start//batch_size + 1} completed")
        print(f"   ✅ Success: {success_count}")
        print(f"   ⚠️ No GDrive links: {no_gdrive_count}")
        print(f"   ❌ Errors: {error_count}")
        
        # Pause between batches
        if batch_end < len(existing_books):
            delay = input(f"\n⏸️ Pause before next batch? Enter seconds (default: 5): ").strip()
            delay_time = int(delay) if delay.isdigit() else 5
            print(f"⏱️ Waiting {delay_time} seconds...")
            time.sleep(delay_time)
    
    # Final summary
    print(f"\n🎉 LEGACY CONVERSION COMPLETED!")
    print("=" * 50)
    print(f"📂 Total books processed: {processed_count}")
    print(f"✅ Successfully converted: {success_count}")
    print(f"⚠️ No Google Drive links: {no_gdrive_count}")
    print(f"❌ Errors encountered: {error_count}")
    print(f"📈 Success rate: {(success_count/max(processed_count-no_gdrive_count,1))*100:.1f}%")
    
    if test_mode:
        print(f"\n🧪 TEST MODE SUMMARY:")
        print(f"   📋 Changes previewed, no commits made")
        print(f"   🔄 Run again without test mode to apply changes")
    else:
        if success_count > 0:
            print(f"\n🚀 GitHub Pages will rebuild automatically")
            print(f"🔗 {success_count} books now have {selected_platform['name']} links")
            print(f"📱 Platform index {selected_platform['index']} ready for API configuration")
    
    print(f"\n📊 Platform Statistics:")
    print(f"   🆔 Platform ID: {selected_platform['id']}")
    print(f"   📍 Platform Index: {selected_platform['index']}")
    print(f"   🎯 Books Updated: {success_count}")
    print(f"   ⏱️ Total Processing Time: ~{processed_count * 3} seconds")

# Run Legacy Conversion
legacy_book_conversion()
